In [1]:
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, DistilBertModel
print('Loading base model')
base_model = DistilBertModel.from_pretrained('distilbert-base-cased')
print("Loading classification model from base model's checkpoint")
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

Loading base model
Loading classification model from base model's checkpoint


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from transformers import DistilBertTokenizer, DistilBertTokenizerFast, AutoTokenizer
name = "distilbert/distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(name)


In [3]:
# Load data
from datasets import load_dataset, DatasetDict



# DataLoader(zip(list1, list2))
dataset_name = "stanfordnlp/imdb"

imdb_dataset = load_dataset(dataset_name)


# Just take the first 50 tokens for speed/running on cpu
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=1111).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=1111).select(range(128, 160)).map(truncate),
)

# Prepare the dataset - this tokenizes the dataset in batches of 16 examples.
small_tokenized_dataset = small_imdb_dataset.map(
    lambda example: tokenizer(example['text'], padding=True, truncation=True), # https://huggingface.co/docs/transformers/pad_truncation
    batched=True,
    batch_size=16
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["text"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")


In [4]:
small_imdb_dataset['train'][:10]

{'text': ["Probably Jackie Chan's best film in the 1980s, and the one that put him on the map. The scale of this self-directed police drama is evident from the opening and closing scenes, during which a squatters' village and shopping mall are demolished. There are, clearly, differences between the original Chinese",
  'A wonderful movie! Anyone growing up in an Italian family will definitely see themselves in these characters. A good family movie with sadness, humor, and very good acting from all. You will enjoy this movie!! We need more like it.',
  'HORRENDOUS! Avoid like the plague. I would rate this in the top 10 worst movies ever. Special effects, acting, mood, sound, etc. appear to be done by day care students...wait, I have seen programs better than this. Opens like a soft porn show with a blurred nude female doing a',
  'And I absolutely adore Isabelle Blais!!! She was so cute in this movie, and far different from her role in "Quebec-Montreal" where she was more like a man-eat

In [5]:
small_tokenized_dataset['train'][0:2]

{'labels': tensor([1, 1]),
 'input_ids': tensor([[  101, 10109,  9662, 10185,   112,   188,  1436,  1273,  1107,  1103,
           3011,   117,  1105,  1103,  1141,  1115,  1508,  1140,  1113,  1103,
           4520,   119,  1109,  3418,  1104,  1142,  2191,   118,  2002,  2021,
           3362,  1110, 10238,  1121,  1103,  2280,  1105,  5134,  4429,   117,
           1219,  1134,   170,  4816,  6718, 18899,   112,  1491,  1105,  6001,
           8796,  1132,  6515,   119,  1247,  1132,   117,  3817,   117,  5408,
           1206,  1103,  1560,  1922,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101,   138,  7310,  2523,   106, 15859,  2898,  1146,  1107,  1126,
           2169,  1266,  1209,  5397,  1267,  2310,  1107,  1292,  2650,   119,
            138,  1363,  1266,  2523,  1114, 12928,   1

In [6]:
# Prepare the dataset - this tokenizes the dataset in batches of 16 examples.
small_tokenized_dataset = small_imdb_dataset.map(
    lambda example: tokenizer(example['text'], padding=True, truncation=True), # https://huggingface.co/docs/transformers/pad_truncation
    batched=True,
    batch_size=16
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["text"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=16)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

In [8]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params} all model parameters: {all_model_params} percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [9]:
print_number_of_trainable_model_parameters(model)

'trainable model parameters: 65783042 all model parameters: 65783042 percentage of trainable model parameters: 100.00%'

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import (
    LoraConfig, 
    get_peft_model, 
    TaskType,
    PeftModel
)
import numpy as np


lora_config = LoraConfig(
    r=8, # Rank Number
    lora_alpha=32, # Alpha (Scaling Factor)
    lora_dropout=0.05, # Dropout Prob for Lora
    target_modules=["q_lin", "k_lin","v_lin"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
    bias='none',
    task_type=TaskType.SEQ_CLS # Seqence to Classification Task
)

lora_config_1 = LoraConfig(
    r=8, # Rank Number
    lora_alpha=16, # Alpha (Scaling Factor)
    lora_dropout=0.01, # Dropout Prob for Lora
    target_modules=["q_lin", "k_lin","v_lin"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
    bias='none',
    task_type=TaskType.SEQ_CLS # Seqence to Classification Task
)

lora_config_2 = LoraConfig(
    r=16, # Rank Number
    lora_alpha=8, # Alpha (Scaling Factor)
    lora_dropout=0.002, # Dropout Prob for Lora
    target_modules=["q_lin", "k_lin","v_lin"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
    bias='none',
    task_type=TaskType.SEQ_CLS # Seqence to Classification Task
)

peft_model = get_peft_model(model, lora_config_2)



In [11]:
print_number_of_trainable_model_parameters(peft_model)

'trainable model parameters: 1034498 all model parameters: 66817540 percentage of trainable model parameters: 1.55%'

In [12]:
arguments = TrainingArguments(
    output_dir="./result",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=3e-4,
    load_best_model_at_end=True,
    seed=224,
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    #model=model,
    model=peft_model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/teddy3090/miniconda3/envs/hw/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
for batch in train_dataloader:
    input_ids = batch["input_ids"]
    labels = batch["labels"]
    print("Input shape:", input_ids.shape)
    print("Labels shape:", labels.shape)
    break

Input shape: torch.Size([16, 92])
Labels shape: torch.Size([16])


In [14]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.669915,0.593750
2,No log,0.653657,0.656250
3,No log,0.656495,0.593750
4,No log,0.620513,0.718750
5,No log,0.579565,0.750000
6,No log,0.530979,0.750000
7,No log,0.489540,0.750000
8,No log,0.465196,0.718750
9,No log,0.439610,0.781250
10,No log,0.429755,0.781250


/home/teddy3090/miniconda3/envs/hw/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/teddy3090/miniconda3/envs/hw/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/teddy3090/miniconda3/envs/hw/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/teddy3090/miniconda3/envs/hw/lib/python3.10/site-packages/huggingface_

TrainOutput(global_step=80, training_loss=0.5521339416503906, metrics={'train_runtime': 8.4197, 'train_samples_per_second': 152.024, 'train_steps_per_second': 9.501, 'total_flos': 32406528232320.0, 'train_loss': 0.5521339416503906, 'epoch': 10.0})

In [15]:
import torch
test_str = "I enjoyed the movie!"

finetuned_model = AutoModelForSequenceClassification.from_pretrained("result/checkpoint-40")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(finetuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

/home/teddy3090/miniconda3/envs/hw/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


POSITIVE


In [16]:
# evaluating the model is very easy

results = trainer.evaluate()                           # just gets evaluation metrics
results = trainer.predict(small_tokenized_dataset['val'])

In [17]:
results

PredictionOutput(predictions=array([[-0.03740241, -0.20690368],
       [ 0.12224149, -0.24344756],
       [-0.17776576, -0.03724482],
       [-0.13465351, -0.02291623],
       [-1.5709504 ,  1.0764621 ],
       [-0.16820702, -0.05394708],
       [ 0.48865572, -0.5074051 ],
       [ 0.39148882, -0.5054668 ],
       [-1.160519  ,  0.7769239 ],
       [-1.251047  ,  0.8715321 ],
       [-0.38490903,  0.13534473],
       [ 0.91885054, -0.9023751 ],
       [ 0.5671619 , -0.5953146 ],
       [-0.37796575,  0.17148209],
       [ 0.5872896 , -0.68405783],
       [ 0.89205205, -0.874074  ],
       [ 0.04627319, -0.20756498],
       [-1.312896  ,  0.91442865],
       [-0.41346422,  0.16097899],
       [-2.0610247 ,  1.4779649 ],
       [ 0.6141318 , -0.70883626],
       [ 0.1030957 , -0.21508381],
       [-0.17825243, -0.00685348],
       [-0.55719244,  0.2561368 ],
       [ 0.02980587, -0.17486508],
       [-0.7932595 ,  0.46205375],
       [-0.8218774 ,  0.4770876 ],
       [-0.78745306,  0.49